# Data Ingestion

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

train_raw_df = fetch_20newsgroups(subset='train', categories=categories)
test_raw_df = fetch_20newsgroups(subset='test', categories=categories)

x_train, x_val, y_train, y_val = train_test_split(np.array(train_raw_df.data), train_raw_df.target, test_size=0.1)
x_test = np.array(test_raw_df.data)
y_test = test_raw_df.target

# x_train = [x_train[:200] for x in x_train]

print('Train:', len(x_train))
print('Val:', len(x_val))
print('Test:', len(x_test))

Train: 2031
Val: 226
Test: 1502


In [2]:
%reload_ext  autoreload
%autoreload 2

import sys, os
def add_aion(curr_path=None):
    if curr_path is None:
        dir_path = os.getcwd()
        target_path = os.path.dirname(dir_path)
        if target_path not in sys.path:
            print('Added %s into sys.path.' % (target_path))
            sys.path.insert(0, target_path)
            
add_aion()

Added /data/jupyter/common into sys.path.


# Model

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/dscoe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
from aion.embeddings.infersent import InferSentEmbeddings

infer_sent_embs = InferSentEmbeddings(word_embeddings_dir='../model/text/stanford/glove/', verbose=20)
infer_sent_embs.load_model(dest_dir='../model/text/facebook/infersent/')

In [5]:
infer_sent_embs.build_vocab(x_train, tokenize=True)

Found 22119(/46170) words with w2v vectors
Vocab size : 22119


In [6]:
x_train_t = infer_sent_embs.encode(x_train, tokenize=True)
x_test_t = infer_sent_embs.encode(x_test, tokenize=True)

/data/jupyter/common/aion/embeddings/infersent_lib/models.py:222: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  sentences[stidx:stidx + bsize]), volatile=True)


In [7]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='newton-cg', max_iter=1000)
model.fit(x_train_t, y_train)

y_pred = model.predict(x_test_t)

/anaconda/envs/py35/lib/python3.5/site-packages/scipy/optimize/linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')


In [8]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print('Accuracy:%.2f%%' % (accuracy_score(y_test, y_pred)*100))
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy:86.55%
Classification Report:
             precision    recall  f1-score   support

          0       0.85      0.76      0.80       319
          1       0.86      0.95      0.91       389
          2       0.95      0.79      0.86       396
          3       0.82      0.94      0.87       398

avg / total       0.87      0.87      0.86      1502

